In [1]:
import numpy as np
import load_utils
import spine_augmentation as aug
import confidence_map as cmap
import spine_model
import torch.optim as optim
import torch.nn as nn
import torch
import os.path as path
import torchvision
import matplotlib.pyplot as plt
import cv2
import torch.nn.functional as F
from PIL import Image
import folders as f
import os
import argparse


In [2]:
net = spine_model.SpineModelPAF()
save_path = f.checkpoint_heat_path
net.load_state_dict(torch.load(save_path))
net.eval()
net.cuda()
test_data_loader = load_utils.test_loader(1,  load_angle=True)
device = torch.device("cuda")
os.makedirs(f.validation_plot_out, exist_ok=True)

## Gaussian to point

In [3]:
def cvshow(img):
    assert len(img.shape)==2
    #img = cv2.resize(img, dsize=None, fx=4, fy=4, interpolation=cv2.INTER_NEAREST)
    cv2.imshow("img", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def cvsave(img, name):
    assert len(img.shape)==2
    cv2.imwrite(path.join(f.validation_plot_out,"{}.jpg".format(name)), img)

In [4]:
def centeroid(heat):
    # Parse center point of connected components
    # Return [p][xy]
    ret, heat = cv2.threshold(heat, 0.99, 1., cv2.THRESH_BINARY)
    heat = np.array(heat*255., np.uint8)
    # num: point number + 1 background
    num, labels = cv2.connectedComponents(heat)
    coords = []
    for label in range(1, num):
        mask = np.zeros_like(labels, dtype=np.uint8)
        mask[labels == label] = 255
        M = cv2.moments(mask)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        coords.append([cX, cY])
    return coords
    

In [5]:
def line_mask(pt1, pt2_list, hw):
    # Return images with a line from pt1 to each pts in pt2_list
    # Return image pixel value range: [0,1], nparray.
    assert len(hw)==2
    zeros = np.zeros([len(pt2_list), hw[0], hw[1]], dtype=np.uint8)
    masks_with_line = [cv2.line(zeros[i_pt2],tuple(pt1),tuple(pt2),255) for i_pt2, pt2 in enumerate(pt2_list)]
    masks_01 = np.array(masks_with_line, dtype=np.float32)/255.
    
    return masks_01

In [6]:
def line_dist(pt1, pt2_list):
    # Return distances of a point to a list of points.
    # Return numpy array
    pt1 = np.array(pt1)
    pt2_list = np.array(pt2_list)
    dist_1d = pt2_list-pt1
    dist_2d = np.linalg.norm(dist_1d, axis=-1)
    return dist_2d

In [7]:
def coincidence_rate(paf, line_masks, distances):
    # Return confidences of a point connects to a list of points
    # Return nparray, range from 0 to around 5 (not 0-1 due to opencv line divides distance not equal to 1.0)
    assert len(paf.shape)==2
    assert len(line_masks.shape)==3
    assert len(distances.shape)==1
    coincidence = line_masks * paf  # [p2_len][h][w]
    co_sum = np.sum(coincidence, axis=(1,2))
    co_rate = co_sum / distances
    return co_rate

In [8]:
def center_coords(lcrc_pcm):
    # Return left center coordinates, right center coordinates.
    assert len(lcrc_pcm.shape)==3, "expected shape: (lr, h, w)"
    assert lcrc_pcm.shape[0]==2, "1st dim of pcm should have 2 elements: l and r"
    lcrc_coord = [centeroid(c) for c in lcrc_pcm[:]]  # lc_coord: [p][xy]
    return lcrc_coord

def coincidence_rate_from_pcm_paf(lcrc_coord, hw, paf):
    # Return confidences nparray with shape: [p1_len][p2_len]
    assert len(np.array(lcrc_coord[0]).shape)==2, "expected shape: (p, xy). length of lc, rc list can be different"
    assert len(hw)==2, "expected shape length: 2 for h and w"
    assert len(paf.shape)==2, "paf shape length should be 2"
    lc_coord, rc_coord = lcrc_coord
    coins = []  # coincidence rate list, shape: [pt1_len][pt2_len]
    for lc_pt in lc_coord[:]:
        p1_masks = line_mask(lc_pt, rc_coord, hw)  #[p2_len][h][w]
        p1_dist = line_dist(lc_pt, rc_coord)
        coin = coincidence_rate(paf, p1_masks,p1_dist)
        coins.append(coin)
    return np.array(coins)

In [29]:
def pairs_with_highest_confidence(coincidence_rate, confidence_lowerbound=3.5):
    # Return: 2 lists contains paired points. e.g.[3,4,5] and [3,4,6] means l3->r3, l4->r4, l6->r6
    pair_l, pair_r = [], []
    args_1d = np.argsort(coincidence_rate, axis=None)
    lc_args, rc_args = np.unravel_index(args_1d, coincidence_rate.shape)
    for i_arg in reversed(range(len(lc_args))):  # reverse: default argsort gives min->max sort, we want max->min results
        al = lc_args[i_arg]  # index of left center list
        ar = rc_args[i_arg]  # index of right center list
        
        if (al not in pair_l) and (ar not in pair_r):  # Best pair among all
            
            # Check if confidence too low (e.g. 2 wrong points at top and bottom).
            # Real pair should have cofidence of around 4.5
            if coincidence_rate[al][ar] > confidence_lowerbound:
                pair_l.append(al)
                pair_r.append(ar)
        else:
            # At least one point already had a better pair.
            pass
    assert len(pair_l)==len(pair_r)
    return (pair_l, pair_r)

In [10]:
def pair_args_to_value(pair_lr_args, lr_coords):
    # Convert pairs of xy args to pairs of xy values
    al, ar = np.array(pair_lr_args, dtype=np.int)[:]
    cl, cr = lr_coords
    # There may be single points with out pair. In that case , lengthes are different, lr_coords can't be converted to a numpy array, 
    # cause vanilla python list error: "only integer scalar arrays can be converted to a scalar index".
    cl, cr = list(map(np.array, [cl, cr]))  

    
    xy_l = cl[al]
    xy_r = cr[ar]
    return xy_l, xy_r

In [11]:
def bone_vectors(pair_lr_value):
    # Return vector of bones (for angle computation)
    # Shape [bone][xy]
    pair_lr_value = np.array(pair_lr_value)
    assert len(pair_lr_value.shape)==3, "shape should be:(lr, bones, xy)"
    assert pair_lr_value.shape[0]==2, "length of first dim should be 2 for l and r"
    l, r = pair_lr_value
    return r-l

In [12]:
def cos_angle(v1, v2):
    assert v1.shape==(2,)
    assert v2.shape==(2,)
    dot = np.dot(v1, v2)
    len1, len2 = list(map(np.linalg.norm, [v1, v2]))
    an_cos = dot/(len1*len2)
    return an_cos

In [13]:
def angle_matrix(bone_vectors):
    # Return angle matrix: A[i][j]
    # Return degree of each 2 vectors, shape: [bone1][bone2]
    bone_vectors = np.array(bone_vectors)
    assert len(bone_vectors.shape)==2, "expected shape: (bone, xy)"

    num_bones = bone_vectors.shape[0]
    an_matrix = np.zeros((num_bones, num_bones))
    for i in range(num_bones):
        for j in range(num_bones):
            v1, v2=bone_vectors[i], bone_vectors[j]
            an_cos = cos_angle(v1, v2)
            an_matrix[i, j] = an_cos
    # cos_angle some times larger than 1 due to numerical precision
    an_matrix = np.clip(an_matrix, a_min=-1., a_max=1.)
    an_matrix = np.arccos(an_matrix)
    an_matrix = np.rad2deg(an_matrix)
    return an_matrix

In [14]:
def draw_pairs(lr_values,heat_hw, img):
    # Draw the line between pairs on image
    assert len(np.asarray(lr_values).shape)==3, "shape: (lr, p, xy)"
    assert len(img.shape)==2, "shape: (h,w)"
    lv, rv = lr_values
    draw_layer = np.zeros(heat_hw, dtype=np.uint8)
    for i in range(len(lv)):
        pt1 = lv[i]
        pt2 = rv[i]
        cv2.line(draw_layer, tuple(pt1), tuple(pt2), 255)
        cv2.putText(draw_layer, str(i), tuple(pt2), cv2.FONT_HERSHEY_SIMPLEX, 0.25, 255)
    draw_layer = cv2.resize(draw_layer, tuple(reversed(img.shape)), interpolation=cv2.INTER_NEAREST)
    img = np.maximum(img, draw_layer)
    return img

In [15]:
def make_ind1_upper(ind1, ind2, pair_lr_value):
    # Check if ind1 is upper bone. If not, exchange ind1 and 2
    assert len(np.array(pair_lr_value).shape)==3
    left1 = pair_lr_value[0][ind1][1]  #  Relies on leftpoint, y coord
    left2 = pair_lr_value[0][ind2][1] 
    if left2 > left1:  # ind2 is lower
        pass
    else:  # ind1 is lower
        temp = left1
        left1 = left2
        left2 = temp
    return ind1, ind2


In [16]:
def cobb_angles(np_pcm, np_paf, img=None):
    # Return np array of [a1, a2, a3]
    assert len(np_pcm.shape)==3, "expected shape: (c,h,w)"
    assert np_pcm.shape[0]==2, "expect 2 channels at dim 0 for l and r"
    assert len(np_paf.shape)==3, "expected shape: (c,h,w)"
    assert np_paf.shape[0]==1, "expect 1 channel at dim 0 for paf"
    heat_hw = np_pcm.shape[1:3]
    # [lr][xy] coordinate values
    lcrc_coords = center_coords(np_pcm)
    # [p1_len][p2_len] coincidence rate of a point to another point
    coins = coincidence_rate_from_pcm_paf(lcrc_coords, heat_hw, np_paf[0])
    # [lr][p_len] pairs of points. length is the shorter one in length of l,r
    pair_lr = pairs_with_highest_confidence(coins)
    # [lr][p_len][xy], coordinates. (sorted by bone confidence, not up to bottom)
    pair_lr_value = pair_args_to_value(pair_lr, lcrc_coords)
    # [p_len][xy] vector coordinates. (sorted by bone confidence, not up to bottom)
    bones = bone_vectors(pair_lr_value)
    # [len_b][len_b] angle matrix
    am = angle_matrix(bones)
    sort_indices = np.unravel_index(np.argsort(am, axis=None), am.shape)
    # Two indices that composed the largest angle
    max_ind1, max_ind2 = sort_indices[0][-1], sort_indices[1][-1] 
    # Find out which one is upper bone
    max_ind1, max_ind2 = make_ind1_upper(max_ind1, max_ind2, pair_lr_value)
    a1 = am[max_ind1, max_ind2]
    a2 = np.rad2deg(np.arccos(cos_angle(bones[max_ind1], np.array([1, 0]))))
    a3 = np.rad2deg(np.arccos(cos_angle(bones[max_ind2], np.array([1, 0]))))
    # print(a1,  a2, a3)
    # print(max_ind1, max_ind2)
    if img is not None:
        assert len(img.shape)==2, "expected shape: (h,w)"
        plot_img = draw_pairs(pair_lr_value, heat_hw, img)
        return np.array([a1, a2, a3]), plot_img
    else:
        return np.array([a1, a2, a3])

Traverse all test images

In [35]:
def run_on_submit_test():
    ###############
    # Use trainval checkpoint
    ###############
    import csv
    result_name_an123 = []  # Parsing results to be wrote
    submit_example = path.join(f.submit_test_img, "sample_submission.csv")
    with open(submit_example, 'r') as example:
        reader = csv.reader(example)
        example_content = list(reader)
        result_name_an123.append(example_content[0])  # Title line
        name_an123 = example_content[1:]  # Exclude first title line "name, an1, an2, an3"

    net_heat = spine_model.SpineModelPAF()
    net_heat.cuda()
    net_heat.eval()

    save_path = f.checkpoint_heat_trainval_path
    net_heat.load_state_dict(torch.load(save_path))

    filename_list = list(zip(*name_an123))[0]
    for filename in filename_list:
        resize_filename = path.join(f.resize_submit_test_img, filename + ".jpg")
        np_img_ori = cv2.imread(resize_filename, cv2.IMREAD_GRAYSCALE)
        np_img = [[np_img_ori ]]  # NCHW
        np_img = np.asarray(np_img, np.float32)

        np_norm_img = np_img / 255.
        t_norm_img = torch.from_numpy(np_norm_img).to(device)
        with torch.no_grad():
            out_pcm, out_paf, _, _ = net_heat(t_norm_img)
            
        np_pcm = out_pcm.detach().cpu().numpy()
        np_paf = out_paf.detach().cpu().numpy()
        
        pred_angles, plot_img = cobb_angles(np_pcm[0, 4:6], np_paf[0], np_img_ori)
        
        result_line = [filename, float(pred_angles[0]), float(pred_angles[1]), float(pred_angles[2])]
        result_name_an123.append(result_line)
        print(filename)
        cvsave(plot_img, "{}".format(filename))
    
    with open(path.join(f.data_root, "submit_result.csv"), "w+", newline='') as result_csv_file:
        writer = csv.writer(result_csv_file)
        [writer.writerow(l) for l in result_name_an123]

In [56]:
# Run on validation set
for step in range(128):
        test_imgs, test_labels, test_angles = next(test_data_loader)
        test_imgs_f = np.asarray(test_imgs, np.float32)[:, np.newaxis, :, :]
        test_imgs_01 = test_imgs_f / 255.0
        test_imgs_tensor = torch.from_numpy(test_imgs_01).cuda()
        with torch.no_grad():
            out_pcm, out_paf, _, _ = net(test_imgs_tensor)  # NCHW
        np_pcm = out_pcm.detach().cpu().numpy()
        np_paf = out_paf.detach().cpu().numpy()
        
        pred_angles, plot_img = cobb_angles(np_pcm[0, 4:6], np_paf[0], test_imgs[0])
        print(step, test_angles[0] - pred_angles)
        cvsave(plot_img, "{}".format(step))
        

0 [ 0.01671131 13.03595478 -1.98824347]
1 [ 3.18664169 -2.56294882 18.88559051]
2 [-2.03720361 -0.1643959  -4.45040771]
3 [-4.19219461  6.75240949 -9.1876041 ]
4 [-0.98644708  3.0656041  -3.20505118]
5 [ -5.61797372 -15.91629051   7.23861679]
6 [ -2.29965528 -15.4126041    3.77994882]
7 [ -1.45065804 -22.24206753   8.49340949]
8 [-1.45334472 -0.1863959  -1.53394882]
9 [-4.04065804 -5.10359051 -4.77506753]
10 [-4.49397769 -6.68504522 -0.44093247]
11 [  4.91386495 -14.02306753   4.41893247]
12 [ 11.54325468 -16.24867779  13.44493247]
13 [ 3.09465001 -8.80610652 12.77375653]
14 [ 1.75494466 -7.14210652 11.04705118]
15 [ 3.78054529 -4.76104522  7.80959051]
16 [-0.89969704 -6.22259051  8.09289348]
17 [-2.81707949 -4.71348897  3.53640949]
18 [ -1.42844112 -19.62798522  -7.7983959 ]
19 [  0.30794466 -14.42194882 -15.19960652]
20 [-10.15818103 -23.19859051  12.58240949]
21 [-5.068      20.09205118  7.49594882]
22 [  7.07871131 -13.56162347  17.88695478]
23 [-2.9580964  -8.16805118 -1.16804522]

In [36]:
run_on_submit_test()

01-July-2019-1
01-July-2019-10
01-July-2019-11
01-July-2019-12
01-July-2019-13
01-July-2019-14
01-July-2019-15
01-July-2019-16
01-July-2019-17
01-July-2019-18
01-July-2019-19
01-July-2019-2
01-July-2019-20
01-July-2019-21
01-July-2019-22
01-July-2019-23
01-July-2019-24
01-July-2019-25
01-July-2019-26
01-July-2019-27
01-July-2019-28
01-July-2019-29
01-July-2019-3
01-July-2019-30
01-July-2019-31
01-July-2019-32
01-July-2019-33
01-July-2019-34
01-July-2019-35
01-July-2019-36
01-July-2019-37
01-July-2019-38
01-July-2019-39
01-July-2019-4
01-July-2019-40
01-July-2019-41
01-July-2019-42
01-July-2019-43
01-July-2019-44
01-July-2019-45
01-July-2019-46
01-July-2019-47
01-July-2019-48
01-July-2019-49
01-July-2019-5
01-July-2019-50
01-July-2019-51
01-July-2019-52
01-July-2019-53
01-July-2019-54
01-July-2019-55
01-July-2019-56
01-July-2019-57
01-July-2019-58
01-July-2019-59
01-July-2019-6
01-July-2019-60
01-July-2019-61
01-July-2019-62
01-July-2019-63
01-July-2019-64
01-July-2019-65
01-July-2019-6